In [0]:
table_name = []
temp = []

for i in dbutils.fs.ls('mnt/silver/'):
    temp.append(i.name.split('/')[0])

for i in temp:
    table_name.append(i.split('_')[1])


In [0]:
table_name

Out[2]: ['CountryRegionCurrency',
 'CreditCard',
 'Currency',
 'CurrencyRate',
 'Customer',
 'PersonCreditCard',
 'SalesOrderDetail',
 'SalesOrderHeader',
 'SalesOrderHeaderSalesReason',
 'SalesPerson',
 'SalesPersonQuotaHistory',
 'SalesReason',
 'SalesTaxRate',
 'SalesTerritory',
 'SalesTerritoryHistory',
 'ShoppingCartItem',
 'SpecialOffer',
 'SpecialOfferProduct',
 'Store']

In [0]:
# reading the files from the silver folder and taking all the columns of a table into a list
for name in table_name:
    path = '/mnt/silver/' + 'Sales_' + name
    #print(path)
    df = spark.read.format('delta').load(path)

    column = df.columns

#iterating over all the columns in the list, renaming the column with appropriate format
    for old_col_name in column:
        new_col_name = "".join(["_" + char if char.isupper() and not old_col_name[i-1].isupper() else char for i, char in enumerate(old_col_name)]).lstrip("_")
        df = df.withColumnRenamed(old_col_name, new_col_name)

#once the transformation is done, we then upload all the files to silver layer in the delta format 
    output_path = '/mnt/gold/' + 'Sales_' + name
    df.write.format('delta').mode("overwrite").save(output_path)

In [0]:
df.limit(10).show()

+------------------+--------------------+---------------+--------------------+--------------------+-------------+
|Business_Entity_ID|                Name|Sales_Person_ID|        Demographics|             rowguid|Modified_Date|
+------------------+--------------------+---------------+--------------------+--------------------+-------------+
|               292|Next-Door Bike Store|            279|<StoreSurvey xmln...|a22517e3-848d-4eb...|   2014-09-12|
|               294|Professional Sale...|            276|<StoreSurvey xmln...|b50ca50b-c601-4a1...|   2014-09-12|
|               296|      Riders Company|            277|<StoreSurvey xmln...|337c3688-1339-4e1...|   2014-09-12|
|               298|  The Bike Mechanics|            275|<StoreSurvey xmln...|7894f278-f0c8-4d1...|   2014-09-12|
|               300|   Nationwide Supply|            286|<StoreSurvey xmln...|c3fc9705-a8c4-4f3...|   2014-09-12|
|               302|Area Bike Accesso...|            281|<StoreSurvey xmln...|368be6dd-3